# Shipping movements and port links

The relationship between shipping movements for the ports of Belfast, Felixstowe and Milford Heaven are shown below. These show common port links for outbound journeys (as a percentage of total voyages). It can be seen that all three ports serve very different geographic regions. The ships leaving the port of Belfast predominantly sail to destinations within UK waters, with 60% of ships sailing to the ports of Loch Ryan, Birkenhead and Heysham. Ships leaving the port of Felixstowe generally travel to ports within the continental mainland with nearly 70% terminating at the ports of Rotterdam, Antwerp, Hamburg, Bremerhaven and Amsterdam. Milford Heaven almost exclusively serves international destinations, with 88% of ships sailing to unspecified international ports, New York and Ras Laffen in Qatar. 

In [1]:
# base libraries
import numpy as np
import pandas as pd
import datetime
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# set variable from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

cers_path = config['DEFAULT']['cers_path']
cers_filename = config['DEFAULT']['cers_filename']
port_filename = config['DEFAULT']['port_filename']

In [3]:
# import CERS and port datasets
cers = pd.read_csv(cers_path+cers_filename,header = 0,delimiter = ',')
cers['ETA'] = pd.to_datetime(cers["ETA"])
cers['ETA_date'] = cers['ETA'].dt.date
cers.drop_duplicates(keep = 'last',inplace = True)

ports = pd.read_csv(cers_path+port_filename,header = 0,delimiter = ',')

In [4]:
cers = cers.merge(ports[['LOCODE','Name']], how = 'inner', left_on = 'POC_LOCODE', right_on = 'LOCODE')
cers.drop('LOCODE',axis = 1, inplace=True)
cers.rename(columns={'Name':'POC_name'},inplace=True)
cers = cers.merge(ports[['LOCODE','Name']], how = 'left', left_on = 'next_port_LOCODE', right_on = 'LOCODE')
cers.rename(columns={'Name':'next_port_name'},inplace=True)

In [5]:
port_lookup = {'ZZUKN':'Unknown Int','GBGRK':'Greenock','NLRTM':'Rotterdam','NLRTM':'Antwerp','DEHAM':'Hamburg',
               'DEBRV':'Bremerhaven','BEANR':'Amsterdam','USNYC':'New York','QARLF':'Ras Laffen','NLMOE':'Moerdijk'}

cers['next_port_name' ] = cers.apply(lambda row: (row['next_port_LOCODE'] if pd.isnull(row['next_port_name']) else row['next_port_name']),axis=1)
cers['next_port_name' ] = cers['next_port_name'].replace(port_lookup)

In [6]:
# get the significant links for Belfast
belfast = cers[cers.POC_name == 'Belfast'].copy()
belfast = belfast.groupby('next_port_name')['voyage_id'].count()
belfast = belfast/belfast.sum()
belfast = (belfast*100).astype(int)
belfast = pd.DataFrame(belfast)
belfast.sort_values(ascending=False, by = 'voyage_id').head()

,voyage_id
next_port_name,
Loch Ryan Pt,35
Birkenhead,15
Heysham,9
Unknown Int,5
Liverpool,2


In [7]:
# get the significant links for Felixstowe
felix = cers[cers.POC_name == 'Felixstowe'].copy()
felix = felix.groupby('next_port_name')['voyage_id'].count()
felix = felix/felix.sum()
felix = (felix*100).astype(int)
felix = pd.DataFrame(felix)
felix.sort_values(ascending=False, by = 'voyage_id').head()

,voyage_id
next_port_name,
Antwerp,51
Amsterdam,9
Hamburg,7
Bremerhaven,4
Teesport,3


In [8]:
# get the significant links for Milford Heaven
milford = cers[cers.POC_name == 'Milford Haven'].copy()
milford = milford.groupby('next_port_name')['voyage_id'].count()
milford = milford/milford.sum()
milford = (milford*100).astype(int)
milford = pd.DataFrame(milford)
milford.sort_values(ascending=False, by = 'voyage_id').head()

,voyage_id
next_port_name,
Unknown Int,86
GB888,4
Antwerp,1
CAMTR,0
CAQUE,0
